在上一教程中，我们介绍了Tensor以及对其的操作。在本教程中，我们将介绍自动微分，这是优化机器学习模型的关键技术。

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

渐变胶带<br>
TensorFlow提供了tf.GradientTape API用于自动区分-计算计算相对于其输入变量的梯度。Tensorflow将tf.GradientTape在a 上下文内执行的所有操作“记录” 到“ tape”上。然后，Tensorflow使用该磁带和与每个已记录操作相关联的梯度来使用反向模式微分来计算``已记录''计算的梯度。

例如：

In [3]:
x

<tf.Tensor: id=2, shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

In [4]:
z

<tf.Tensor: id=5, shape=(), dtype=float32, numpy=16.0>

In [2]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

# Derivative of z with respect to the original input tensor x
dz_dx = t.gradient(z, x)
for i in [0, 1]:
    for j in [0, 1]:
        assert dz_dx[i][j].numpy() == 8.0

您还可以请求相对于在“记录的” tf.GradientTape上下文中计算的中间值的输出梯度。

In [3]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

# Use the tape to compute the derivative of z with respect to the
# intermediate value y.
dz_dy = t.gradient(z, y)
assert dz_dy.numpy() == 8.0

默认情况下，只要调用GradientTape.gradient（）方法，就会释放GradientTape拥有的资源。要在同一计算上计算多个梯度，请创建一个persistent梯度带。gradient()当磁带对象被垃圾回收时，释放资源时，这允许对方法的多次调用。例如：

In [4]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = x * x
    z = y * y
dz_dx = t.gradient(z, x)  # 108.0 (4*x^3 at x = 3)
dy_dx = t.gradient(y, x)  # 6.0
del t  # Drop the reference to the tape

记录控制流程<br>
因为磁带在执行时记录操作，所以自然会处理Python控制流（例如，使用ifs和whiles）：

In [5]:
def f(x, y):
    output = 1.0
    for i in range(y):
        if i > 1 and i < 5:
            output = tf.multiply(output, x)
    return output

def grad(x, y):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x, y)
    return t.gradient(out, x)

x = tf.convert_to_tensor(2.0)

assert grad(x, 6).numpy() == 12.0
assert grad(x, 5).numpy() == 12.0
assert grad(x, 4).numpy() == 4.0

高阶渐变<br>
GradientTape记录上下文管理器内部的操作以进行自动区分。如果在那种情况下计算梯度，那么也将记录梯度计算。因此，完全相同的API也适用于高阶渐变。例如：

In [6]:
x = tf.Variable(1.0)  # Create a Tensorflow variable initialized to 1.0

with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y = x * x * x
  # Compute the gradient inside the 't' context manager
  # which means the gradient computation is differentiable as well.
    dy_dx = t2.gradient(y, x)
d2y_dx2 = t.gradient(dy_dx, x)

assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0

下一步<br>
在本教程中，我们介绍了TensorFlow中的梯度计算。这样，我们就具有构建和训练神经网络所需的足够的原语。